## Flights and Domestic Airlines 
Providing the best recommendations for domestic travel in the U.S. In our general applications we are using 2015 data to determine delay information and predict which airlines contains the best flights. 

We are also determining if the weather that day was the reason for the flight cancellation or weather the airline was at fault.

Finally with flight. We are looking into Hotel Recommendations and determining the best hotels domestically based on reviews.


In [40]:
# Install mysql connector, SQLAlchemy
import pandas as pd; import numpy as np, os, mysql.connector as maria; from sqlalchemy import create_engine
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Time, DateTime, Date 

In [28]:
# Get Flight Delay information and cleanse the data for missing values

# Airports CSV
airport_info = pd.read_csv("airports.csv", sep = ",")

# Airlines CSV
airlines_info = pd.read_csv("airlines.csv", sep = ",")
airlines_info

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways
5,OO,Skywest Airlines Inc.
6,AS,Alaska Airlines Inc.
7,NK,Spirit Air Lines
8,WN,Southwest Airlines Co.
9,DL,Delta Air Lines Inc.


In [29]:
airport_info

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447
5,ACK,Nantucket Memorial Airport,Nantucket,MA,USA,41.25305,-70.06018
6,ACT,Waco Regional Airport,Waco,TX,USA,31.61129,-97.23052
7,ACV,Arcata Airport,Arcata/Eureka,CA,USA,40.97812,-124.10862
8,ACY,Atlantic City International Airport,Atlantic City,NJ,USA,39.45758,-74.57717
9,ADK,Adak Airport,Adak,AK,USA,51.87796,-176.64603


In [4]:
import pandas as pd
# Flight information (5,819,079  Rows)
my_chunk = pd.read_csv("flights.csv", sep=',', nrows =1000000) # Read Rows in Smaller Numbers or chunks

columns = list(my_chunk) # Create a list of column names for the dataset

my_chunk.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Cleanse Data
# Concatenate Date columns and give day of the week (e.g. 1/1/2015 and Sunday instead of 7)
my_chunk['DAY'] + "-" + my_chunk['MONTH'] + "-" + my_chunk['YEAR'] # combine values and change into STRPARSE TIME Format
# concatenate the airline and flight number (e.g. AA1738)

# Change the times in the dataframe so they are formatted as HH:MM (e.g. 23:54 instead of 2354)



In [49]:
# Data Storage
# Create Table that will store flight data in MariaDB

meta = MetaData()

# Run this cell once
try:
    flights = Table(
       'flights', meta, 
       Column('YEAR', Integer)
    )

    meta.create_all(engine)

    def add_column(engine, table_name, column):
        column_name = column.compile(dialect=engine.dialect)
        column_type = column.type.compile(engine.dialect)
        engine.execute('ALTER TABLE %s ADD COLUMN %s %s' % (table_name, column_name, column_type))

    dtypes = [Date, String(50), String(50),String(6),String(3),String(3), Time, Time, Integer
              , Integer, Time, Integer, Integer, Integer, Integer, Time, Integer, Time, Time, Integer, String(50)
              , String(50), String(1), Integer, Integer, Integer, Integer, Integer]

    columns = list(my_chunk)
    columns = columns[3:]
    columns[0] = "DATE"

    for c,d in zip(columns,dtypes):
        column = Column(c,d)
        add_column(engine, "flights", column)
except:
    print("The Database already has the columns!")


# Import the first 100k rows into database for further analysis
#data.to_sql(name='flights', con=engine, if_exists = 'append', index=False)

Database already has the columns


In [ ]:
# Create Machine Learning Model to predict if Airline probability of having a delay (SciKit Learn Values)

In [ ]:
# Use Weather API to see if weather affected the flight status (e.g. Weatherbug)

In [ ]:
# Create Visualizations of Airport information